In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

In [2]:
#Path for data files
employee_path = "CSV Files/Resources/employee_data.csv"
insurance_path = "CSV Files/Resources/insurance_data.csv"
vendor_path = "CSV Files/Resources/vendor_data.csv"

#Read the csv files
employee_file = pd.read_csv(employee_path)
insurance_file = pd.read_csv(insurance_path)
vendor_file = pd.read_csv(vendor_path)

#Combine files
ins_claim_fraud_data = [employee_file, insurance_file, vendor_file]
ins_claim_fraud = employee_file.merge(insurance_file, on= ['ADDRESS_LINE1', 'ADDRESS_LINE2', 'CITY', 'STATE']).merge(vendor_file, on= ['ADDRESS_LINE1', 'ADDRESS_LINE2', 'CITY', 'STATE'])
ins_claim_fraud.head()

,AGENT_ID_x,AGENT_NAME,DATE_OF_JOINING,ADDRESS_LINE1,ADDRESS_LINE2,CITY,STATE,POSTAL_CODE_x,EMP_ROUTING_NUMBER,EMP_ACCT_NUMBER,...,ANY_INJURY,POLICE_REPORT_AVAILABLE,INCIDENT_STATE,INCIDENT_CITY,INCIDENT_HOUR_OF_THE_DAY,AGENT_ID_y,VENDOR_ID_x,VENDOR_ID_y,VENDOR_NAME,POSTAL_CODE
0,AGENT00010,Bernard Haug,9/4/1992,2278 North Windswept Lane,NaN,Fayetteville,AR,72703,75517997,ADZX82914131196683,...,0,1,OK,Oklahoma City,5,AGENT00696,VNDR00570,VNDR00310,Salinas-Ferrell,72703
1,AGENT00010,Bernard Haug,9/4/1992,2278 North Windswept Lane,NaN,Fayetteville,AR,72703,75517997,ADZX82914131196683,...,1,0,TN,Nashville,5,AGENT00564,VNDR00193,VNDR00310,Salinas-Ferrell,72703
2,AGENT00010,Bernard Haug,9/4/1992,2278 North Windswept Lane,NaN,Fayetteville,AR,72703,75517997,ADZX82914131196683,...,1,0,CA,King City,11,AGENT00499,NaN,VNDR00310,Salinas-Ferrell,72703
3,AGENT00010,Bernard Haug,9/4/1992,2278 North Windswept Lane,NaN,Fayetteville,AR,72703,75517997,ADZX82914131196683,...,1,1,AL,Pelham,10,AGENT01078,NaN,VNDR00310,Salinas-Ferrell,72703
4,AGENT00012,Jeffrey Forbis,8/5/2010,2619 North Quality Lane,#315,Fayetteville,AR,72703,121484709,IQRF67979220911116,...,1,0,KY,Louisville,15,AGENT01104,NaN,VNDR00031,Johnson Ltd,72703


In [3]:
ins_claim_fraud.tail()

,AGENT_ID_x,AGENT_NAME,DATE_OF_JOINING,ADDRESS_LINE1,ADDRESS_LINE2,CITY,STATE,POSTAL_CODE_x,EMP_ROUTING_NUMBER,EMP_ACCT_NUMBER,...,ANY_INJURY,POLICE_REPORT_AVAILABLE,INCIDENT_STATE,INCIDENT_CITY,INCIDENT_HOUR_OF_THE_DAY,AGENT_ID_y,VENDOR_ID_x,VENDOR_ID_y,VENDOR_NAME,POSTAL_CODE
582,AGENT01172,Katherine Garcia,1/3/2017,9223 Elgin Circle,NaN,Anchorage,AK,99502,30474525,JMTY84637120051395,...,0,1,CA,Fremont,3,AGENT01005,VNDR00350,VNDR00450,"Green, Perry and Anderson",99502
583,AGENT01172,Katherine Garcia,1/3/2017,9223 Elgin Circle,NaN,Anchorage,AK,99502,30474525,JMTY84637120051395,...,1,1,AZ,Glendale,12,AGENT00847,VNDR00006,VNDR00450,"Green, Perry and Anderson",99502
584,AGENT01192,Maria Fitzgerald,5/13/2010,2236 Susitna Drive,NaN,Anchorage,AK,99517,57730721,WMQO78155993850287,...,0,1,DC,Washington,16,AGENT00601,VNDR00565,VNDR00421,Russo Inc,99517
585,AGENT01192,Maria Fitzgerald,5/13/2010,2236 Susitna Drive,NaN,Anchorage,AK,99517,57730721,WMQO78155993850287,...,1,1,CA,Oakland,23,AGENT00219,VNDR00374,VNDR00421,Russo Inc,99517
586,AGENT01192,Maria Fitzgerald,5/13/2010,2236 Susitna Drive,NaN,Anchorage,AK,99517,57730721,WMQO78155993850287,...,1,0,AZ,Litchfield Park,10,AGENT00227,VNDR00564,VNDR00421,Russo Inc,99517


In [4]:
incident_cities_unique = len(ins_claim_fraud["INCIDENT_CITY"].unique())
incident_cities_unique

126

In [5]:
incident_count = ins_claim_fraud["INCIDENT_CITY"].value_counts()
incident_count

INCIDENT_CITY
Manchester         37
Louisville         36
Arvada             33
Montgomery         33
Fayetteville       32
                   ..
Guilford            1
Rancho Cordova      1
Charlotte           1
Wheelock            1
Litchfield Park     1
Name: count, Length: 125, dtype: int64

In [6]:
incident_state_unique = len(ins_claim_fraud["INCIDENT_STATE"].unique())
incident_state_unique

16

In [7]:
incident_state_count = ins_claim_fraud["INCIDENT_STATE"].value_counts()
incident_state_count


INCIDENT_STATE
CA    72
FL    52
CO    42
KY    39
CT    37
AL    35
GA    35
MD    34
OK    33
TN    33
AR    33
MA    32
AZ    30
VT    28
DC    26
AK    26
Name: count, dtype: int64

In [8]:
incident_hour_unique = len(ins_claim_fraud["INCIDENT_HOUR_OF_THE_DAY"].unique())
incident_hour_unique

24

In [9]:
incident_hour_count = ins_claim_fraud["INCIDENT_HOUR_OF_THE_DAY"].value_counts()
incident_hour_count

INCIDENT_HOUR_OF_THE_DAY
10    46
12    33
18    31
13    30
11    29
0     29
16    28
22    27
23    26
9     26
21    24
3     23
5     22
1     21
14    21
17    20
20    20
7     20
15    20
2     20
6     19
19    18
4     18
8     16
Name: count, dtype: int64

In [10]:
vender_unique = len(ins_claim_fraud["VENDOR_NAME"].unique())
vender_unique

163

In [11]:
vender_count = ins_claim_fraud["VENDOR_NAME"].value_counts()
vender_count

VENDOR_NAME
Anderson, Johnson and Kaiser    16
Horne Inc                       12
Blake, Jackson and Scott        12
Richardson, Bates and Olson     10
Jones-Banks                     10
                                ..
Potter Inc                       1
Gray-Murray                      1
Fleming Group                    1
Ewing, Hudson and Davis          1
Ochoa and Sons                   1
Name: count, Length: 163, dtype: int64

In [12]:
police_rep_count = ins_claim_fraud["POLICE_REPORT_AVAILABLE"].value_counts()
police_rep_count

POLICE_REPORT_AVAILABLE
1    412
0    175
Name: count, dtype: int64

In [13]:
injury_count = ins_claim_fraud["ANY_INJURY"].value_counts()
injury_count

ANY_INJURY
1    427
0    160
Name: count, dtype: int64

In [14]:
severity_count = ins_claim_fraud["INCIDENT_SEVERITY"].value_counts()
severity_count

INCIDENT_SEVERITY
Major Loss    206
Total Loss    195
Minor Loss    186
Name: count, dtype: int64

In [15]:
authority_count = ins_claim_fraud["AUTHORITY_CONTACTED"].value_counts()
authority_count

AUTHORITY_CONTACTED
Ambulance    267
Police       140
Other         70
Name: count, dtype: int64

In [16]:
risk_count = ins_claim_fraud["RISK_SEGMENTATION"].value_counts()
risk_count

RISK_SEGMENTATION
L    254
M    231
H    102
Name: count, dtype: int64

In [17]:
claim_status_count = ins_claim_fraud["CLAIM_STATUS"].value_counts()
claim_status_count

CLAIM_STATUS
A    560
D     27
Name: count, dtype: int64

In [18]:
claim_amount_unique = len(ins_claim_fraud["CLAIM_AMOUNT"].unique())
claim_amount_unique

79

In [19]:
claim_amount_count = ins_claim_fraud["CLAIM_AMOUNT"].value_counts()
claim_amount_count

CLAIM_AMOUNT
5000     44
4000     42
2000     37
1000     32
10000    31
         ..
28000     1
60000     1
96000     1
48000     1
35000     1
Name: count, Length: 79, dtype: int64

In [20]:
education_unique = len(ins_claim_fraud["CUSTOMER_EDUCATION_LEVEL"].unique())
education_unique

7

In [21]:
education_count = ins_claim_fraud["CUSTOMER_EDUCATION_LEVEL"].value_counts()
education_count

CUSTOMER_EDUCATION_LEVEL
Bachelor       248
High School    126
College         67
Masters         54
PhD             36
MD              29
Name: count, dtype: int64

In [22]:
employment_count = ins_claim_fraud["EMPLOYMENT_STATUS"].value_counts()
employment_count

EMPLOYMENT_STATUS
Y    533
N     54
Name: count, dtype: int64

In [23]:
tenure_unique = len(ins_claim_fraud["TENURE"].unique())
tenure_unique

109

In [24]:
tenure_count = ins_claim_fraud["TENURE"].value_counts()
tenure_count

TENURE
44     13
99     12
64     12
58     11
87     11
       ..
35      1
27      1
83      1
37      1
114     1
Name: count, Length: 109, dtype: int64

In [25]:
social_class_count = ins_claim_fraud["SOCIAL_CLASS"].value_counts()
social_class_count

SOCIAL_CLASS
MI    261
LI    185
HI    141
Name: count, dtype: int64

In [26]:
house_type_count = ins_claim_fraud["HOUSE_TYPE"].value_counts()
house_type_count

HOUSE_TYPE
Mortgage    233
Rent        199
Own         155
Name: count, dtype: int64

In [27]:
family_size_unique = len(ins_claim_fraud["NO_OF_FAMILY_MEMBERS"].unique())
family_size_unique

6

In [28]:
family_size_count = ins_claim_fraud["NO_OF_FAMILY_MEMBERS"].value_counts()
family_size_count

NO_OF_FAMILY_MEMBERS
7    110
6    105
2    102
4     97
5     87
3     86
Name: count, dtype: int64